In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import os
from splinter import Browser
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\BZ Northwestern BC\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection  from Eli's Stu_Reddit_Scraper for 12.2.4 that he slacked to us at 7:51 on 3/16
db = client.mars_news_db
collection = db.articles
print(f'collection; {collection}')


collection; Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mars_news_db'), 'articles')


Step 1 - Scraping:  NASA Mars News

In [4]:
# URL of page to be scraped
url = 'https://redplanetscience.com/'

# Retrieve page with the requests module
# response = requests.get(url)
browser.visit(url)
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

#print(soup.prettify())

news_title =soup.find("div", "content_title").get_text()
news_body = soup.find("div", "article_teaser_body").get_text()
news_title = news_title.replace(news_body, '')

print(f'news_title: {news_title}')
print(f'news_body: {news_body}')

news_title: NASA's Mars 2020 Rover Goes Coast-to-Coast to Prep for Launch
news_body: The agency's first step in returning rocks from Mars just arrived at Kennedy Space Center. The Mars 2020 team now begins readying for a launch to the Red Planet this July.


In [5]:
browser.quit()

Step 1 - Scraping:  JPL Mars Space Images - Featured Image

In [6]:
browser = Browser("chrome", **executable_path, headless=False)
url = 'https://spaceimages-mars.com/'

# Retrieve page with the visit module
browser.visit(url)
html = browser.html

# Click on "FULL IMAGE" button
browser.links.find_by_partial_text('FULL IMAGE').click()

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())

fade_in_image = soup.find_all("img", "headerimage fade-in")
featured_image_url=url + fade_in_image[0]['src']
print(f'featured_image_url: {featured_image_url}')
print()

featured_image_url: https://spaceimages-mars.com/image/featured/mars1.jpg



In [7]:
browser.quit()

Step 1 - Scraping:  JPL Mars Space Images - Mars Facts

In [8]:
url = 'https://galaxyfacts-mars.com/'

tables = pd.read_html(url)
# tables
# print(type(tables)) #BZ saw it returns list class
# tables
# Get Dataframe of first table
df = tables[0]
print(df.head())

# Convert df to html
html_table = df.to_html()
print(f'html_table: {html_table}')

                         0                1                2
0  Mars - Earth Comparison             Mars            Earth
1                Diameter:         6,779 km        12,742 km
2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
3                   Moons:                2                1
4       Distance from Sun:   227,943,824 km   149,598,262 km
html_table: <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
      <th>2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Mars - Earth Comparison</td>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    

Step 1 - Scraping:  Mars Hemispheres

In [9]:
browser = Browser("chrome", **executable_path, headless=False)
url = 'https://marshemispheres.com/'

# Retrieve page with the visit module
browser.visit(url)

html = browser.html
# print("html")
# print(html)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())

results = soup.find_all('div', class_='item')

i = 0
titles = {}
img_urls = {}
Enhanced_dict= {}
mydict ={}
hemisphere_image_urls = []

# Next look for <h3>
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = results[i].find('h3').text
#         print(f'title within try loop: {title}')
        titles[i]=title
#         print(f'titles (plural): {titles}')
        
        # Run only if title is available
        if (title != ''):
            # Print results
#             print('-------------')
#             print(f"title in title != '': {title}")
            

            # Click on proper enhanced image button
            browser.links.find_by_partial_text(title).click()
#             print("Should sleep for 0.5")
            time.sleep(0.5)

            html = browser.html
            # print("html")
            # print(html)

            # Create BeautifulSoup object; parse with 'html.parser'; find description iterables
            enhanced_soup = BeautifulSoup(html, 'html.parser')
            enhanced_results = enhanced_soup.find_all('div', class_='description')

            # Loop through returned results
            for enhanced_result in enhanced_results:
                # Error handling
                try:
                    link = enhanced_result.a['href']
#                     print(link)
                    img_url=url + link
#                     print(img_url)
                except Exception as e:
                    print(e)
            
#             print(f'title: {title}')
#             print(f'img_url: {img_url}')

    
            #Assigning value to a specific key. This key will be added if its not available already. 
            Enhanced_dict[title] = img_url
            # print(f'Enhanced_dict is now: {Enhanced_dict}')
            
            # Click on back button to return to home page
            browser.links.find_by_partial_text('Back').click()
#             print("Should sleep for 0.5")
            time.sleep(0.5)
            
    except Exception as e:
        print(f'error: {e}')
    i+=1

hemisphere_image_urls = [Enhanced_dict]
print()
print(f'hemisphere_image_urls: {hemisphere_image_urls}')
print()
# print(f'titles: {titles}')
# print(f'img_urls (plural): {img_urls}')
# print("Done")


hemisphere_image_urls: [{'Cerberus Hemisphere Enhanced': 'https://marshemispheres.com/images/cerberus_enhanced.tif', 'Schiaparelli Hemisphere Enhanced': 'https://marshemispheres.com/images/schiaparelli_enhanced.tif', 'Syrtis Major Hemisphere Enhanced': 'https://marshemispheres.com/images/syrtis_major_enhanced.tif', 'Valles Marineris Hemisphere Enhanced': 'https://marshemispheres.com/images/valles_marineris_enhanced.tif'}]



In [10]:
browser.quit()    

Step 2 - MongoDB and Flask Application